In [2]:
from cleanlab_studio import Studio
import pandas as pd
from typing import List, Tuple

In [5]:
# Load log entries dataset
df = pd.read_csv("application_logs.csv")

In [6]:
# Initialize Cleanlab Studio
studio = Studio("YOUR API KEY HERE")

# Use a high quality TLM with GPT-4o
tlm = studio.TLM(quality_preset="high", options={"model": "gpt-4o", "log": ["explanation"]})

log_entry = "2024-08-28 11:51:51 INFO: User profile updated for ID: 18341, fields changed: email"
prompt = [f"Analyze this log entry for GDPR violations: {log_entry}\nViolates GDPR (True/False):"]

result = tlm.try_prompt(prompt)
print(f"GDPR Violation: {result[0]['response']}")
print(f"Trustworthiness Score: {result[0]['trustworthiness_score']}")

Querying TLM... 100%|██████████|

GDPR Violation: To determine whether this log entry might indicate a GDPR violation, we need to consider whether it involves the mishandling or unauthorized disclosure of personal data. Here’s a breakdown of the relevant aspects:

1. **Personal Data**: Email addresses are considered personal data under GDPR, as they can be used to directly identify an individual.

2. **Processing**: The log entry indicates that an email address (a piece of personal data) was changed. Processing of personal data includes actions such as collection, storage, use, and update.

3. **Lawful Basis**: Under GDPR, processing personal data requires a lawful basis, which could be user consent, necessity for contract performance, compliance with legal obligations, protection of vital interests, performance of a task carried out in the public interest, or legitimate interests pursued by the controller.

4. **Security**: The mere fact that an email address was updated is not inherently a GDPR violation. However, GD

In [6]:
def parse_gdpr_violation(response: str) -> bool:
    """
    Extracts the GDPR violation status from the response.
    
    Args:
        response (str): Response from the TLM
    
    Returns:
        bool: True if violates GDPR, False otherwise
    """
    return response.strip().lower() == 'true'

def classify_gdpr_violations(texts: List[str], prompt_template: str) -> Tuple[List[bool], List[float], List[str]]:
    """
    Classifies a list of log entries for GDPR violations using TLM.
    
    Args:
        texts (List[str]): List of log entries to classify
        prompt_template (str): Template string for formatting the prompt
    
    Returns:
        Tuple[List[bool], List[float], List[str]]: A tuple containing three lists:
            - List of GDPR violation statuses for each log entry
            - List of trustworthiness scores for each prediction
            - List of low trustworthiness explanations (empty string if trustworthiness is high)
    """
    prompts = [prompt_template.format(text=text) for text in texts]
    outputs = tlm.try_prompt(prompts)
    
    violations = [parse_gdpr_violation(output['response']) for output in outputs]
    trustworthiness_scores = [output['trustworthiness_score'] for output in outputs]
    low_trust_explanations = [output['log']['explanation'] if output['trustworthiness_score'] < 0.5 else '' for output in outputs]
    
    return violations, trustworthiness_scores, low_trust_explanations

In [7]:
# Define the prompt template
prompt_template = """
Analyze the following application log entry and determine if it violates GDPR by containing or processing personal data without proper safeguards.
Consider things like user identifiers, names, email addresses, IP addresses, or any other personally identifiable information.
Respond with only 'True' if it violates GDPR, or 'False' if it complies with GDPR.

Log entry: {text}

Violates GDPR (True/False):
"""

In [8]:
%%time

# Classify all log entries in batch
violations, trustworthiness_scores, low_trust_explanations = classify_gdpr_violations(df["log_entry"].tolist(), prompt_template)

# Add results to the DataFrame
df['violates_GDPR'] = violations
df['trustworthiness'] = trustworthiness_scores
df['low_trust_explanation'] = low_trust_explanations

Querying TLM... 100%|██████████|

CPU times: user 49.9 s, sys: 12.8 s, total: 1min 2s
Wall time: 2min 20s


In [27]:
import textwrap

# Filter for examples with high trustworthiness scores
high_trust_threshold = 0.8
high_trust_logs = df[df['trustworthiness'] >= high_trust_threshold].sort_values('trustworthiness', ascending=False)

print("\nLog Entries with High Trustworthiness Scores:")
print("=" * 80)

for i, (_, row) in enumerate(high_trust_logs.iterrows()):
    if i >= 10:
        break
    print("\nLog Entry:")
    print(textwrap.fill(row['log_entry'], width=80))
    print(f"\nViolates GDPR: {row['violates_GDPR']}")
    print(f"Trustworthiness score: {row['trustworthiness']}")
    print("-" * 80)


Log Entries with High Trustworthiness Scores:

Log Entry:
2024-08-17 17:48:26 INFO: User 'user250@example.com' changed password

Violates GDPR: True
Trustworthiness score: 0.9998444409832958
--------------------------------------------------------------------------------

Log Entry:
2024-09-05 02:16:10 ERROR: New user registered: Bob Brown, DOB: 1976-01-02

Violates GDPR: True
Trustworthiness score: 0.9998299213052612
--------------------------------------------------------------------------------

Log Entry:
2024-08-28 15:28:47 INFO: New user registered: Jane Brown, DOB: 1985-11-21

Violates GDPR: True
Trustworthiness score: 0.9998298630687377
--------------------------------------------------------------------------------

Log Entry:
2024-08-17 01:10:32 DEBUG: Geo-location data updated for user 'user_140': lat
-71.9892, long 162.0127

Violates GDPR: True
Trustworthiness score: 0.999812477946249
--------------------------------------------------------------------------------

Log Ent

In [10]:

# Filter for GDPR violations with high trustworthiness scores
gdpr_violations = df[(df['violates_GDPR'] == True) & (df['trustworthiness'] >= high_trust_threshold)]

print("\nTop GDPR-violating log entries (high trustworthiness):")
gdpr_violations[['log_entry', 'violates_GDPR', 'trustworthiness']].head()


Top GDPR-violating log entries (high trustworthiness):


,log_entry,violates_GDPR,trustworthiness
9,2024-09-14 22:23:46 DEBUG: Password reset requ...,True,0.998532
15,2024-09-11 16:27:24 INFO: Geo-location data up...,True,0.995860
22,2024-08-30 10:43:05 DEBUG: User login successf...,True,0.993578
23,2024-08-24 01:31:02 WARN: Newsletter subscript...,True,0.999325
28,2024-08-24 19:32:18 WARN: New user registered:...,True,0.996044


In [25]:
import textwrap

# Filter for examples with low trustworthiness scores
low_trust_threshold = 0.5
low_trust_logs = df[(df['violates_GDPR'] == True) & 
                    (df['trustworthiness'] < low_trust_threshold)].sort_values('trustworthiness')

print("\nLog Entries with Low Trustworthiness Scores:")
print("=" * 80)

for _, row in low_trust_logs.iterrows():
    print("\nLog Entry:")
    print(textwrap.fill(row['log_entry'], width=80))
    print(f"\nViolates GDPR: {row['violates_GDPR']}")
    print(f"Trustworthiness score: {row['trustworthiness']}")
    print("\nLow trustworthiness explanation:")
    print(textwrap.fill(row['low_trust_explanation'], width=80))
    print("-" * 80)


Log Entries with Low Trustworthiness Scores:

Log Entry:
2024-09-09 12:25:00 INFO: Unauthorized access attempt to admin panel from IP:
53.125.88.241

Violates GDPR: True
Trustworthiness score: 0.2336285708561434

Low trustworthiness explanation:
The answer "True" indicates that the log entry violates GDPR by containing or
processing personal data without proper safeguards. However, this assessment is
not entirely accurate. The log entry contains an IP address, which can be
considered personal data under GDPR if it can be linked to an individual.
However, the mere presence of an IP address in a log entry does not
automatically constitute a GDPR violation. The context of how the IP address is
used, stored, and protected is crucial. If the IP address is used solely for
security purposes and is adequately protected, it may not violate GDPR.
Therefore, the answer "True" is not definitively correct without additional
context.
-----------------------------------------------------------------

In [12]:
print("\nTop low-trustworthiness log entries:")
low_trust_logs[['log_entry', 'violates_GDPR', 'trustworthiness', 'low_trust_explanation']].head()


Top low-trustworthiness log entries:


,log_entry,violates_GDPR,trustworthiness,low_trust_explanation
432,2024-09-09 12:25:00 INFO: Unauthorized access ...,True,0.233629,"The answer ""True"" indicates that the log entry..."
542,2024-08-30 14:48:51 ERROR: Unauthorized access...,True,0.393348,"The answer provided is ""True,"" indicating that..."
891,2024-08-26 06:07:39 ERROR: Unauthorized access...,True,0.405257,"The answer ""True"" indicates that the log entry..."
954,2024-08-22 00:01:31 ERROR: Unauthorized access...,True,0.406365,"The answer provided is ""True,"" indicating that..."
800,2024-09-06 00:29:17 ERROR: Unauthorized access...,True,0.412790,"The answer ""True"" indicates that the log entry..."


In [13]:
# Calculate and print summary statistics
total_logs = len(df)
gdpr_violations_count = len(gdpr_violations)
low_trust_count = len(low_trust_logs)

print(f"\nSummary:")
print(f"Total log entries analyzed: {total_logs}")
print(f"GDPR violations (high trustworthiness): {gdpr_violations_count} ({gdpr_violations_count/total_logs:.2%})")
print(f"Low trustworthiness entries: {low_trust_count} ({low_trust_count/total_logs:.2%})")


Summary:
Total log entries analyzed: 1000
GDPR violations (high trustworthiness): 239 (23.90%)
Low trustworthiness entries: 8 (0.80%)


In [14]:
# Save the results
df.to_csv("gdpr_analysis_results.csv", index=False)
print("\nResults saved to 'gdpr_analysis_results.csv'")

In [23]:
def display_extreme_trust_examples(df, high_trust_threshold=0.8, low_trust_threshold=0.5):
    """
    Display the highest and lowest trustworthiness examples with wrapped text.
    """
    import textwrap
    
    # Get single highest trust example
    highest_trust = df[df['trustworthiness'] >= high_trust_threshold]\
        .sort_values('trustworthiness', ascending=False)\
        .iloc[0]
    
    # Get single lowest trust example
    lowest_trust = df[(df['violates_GDPR'] == True) & 
                     (df['trustworthiness'] < low_trust_threshold)]\
        .sort_values('trustworthiness')\
        .iloc[0]
    
    print("\nMost Trustworthy Example:")
    print("=" * 80)
    print("\nLog entry:")
    print(textwrap.fill(highest_trust['log_entry'], width=80))
    print(f"\nViolates GDPR: {highest_trust['violates_GDPR']}")
    print(f"Trustworthiness score: {highest_trust['trustworthiness']}")
    print("-" * 80)
    
    print("\nLeast Trustworthy Example:")
    print("=" * 80)
    print("\nLog entry:")
    print(textwrap.fill(lowest_trust['log_entry'], width=80))
    print(f"\nViolates GDPR: {lowest_trust['violates_GDPR']}")
    print(f"Trustworthiness score: {lowest_trust['trustworthiness']}")
    if lowest_trust['low_trust_explanation']:
        print("\nLow trustworthiness explanation:")
        print(textwrap.fill(lowest_trust['low_trust_explanation'], width=80))
    print("-" * 80)

# Call the function
display_extreme_trust_examples(df)


Most Trustworthy Example:

Log entry:
2024-08-17 17:48:26 INFO: User 'user250@example.com' changed password

Violates GDPR: True
Trustworthiness score: 0.9998444409832958
--------------------------------------------------------------------------------

Least Trustworthy Example:

Log entry:
2024-09-09 12:25:00 INFO: Unauthorized access attempt to admin panel from IP:
53.125.88.241

Violates GDPR: True
Trustworthiness score: 0.2336285708561434

Low trustworthiness explanation:
The answer "True" indicates that the log entry violates GDPR by containing or
processing personal data without proper safeguards. However, this assessment is
not entirely accurate. The log entry contains an IP address, which can be
considered personal data under GDPR if it can be linked to an individual.
However, the mere presence of an IP address in a log entry does not
automatically constitute a GDPR violation. The context of how the IP address is
used, stored, and protected is crucial. If the IP address is use